In [15]:
import attr
from unitdoc import UnitDocRegistry

# Unit examples

In [19]:
udr = UnitDocRegistry()

In [20]:
area = udr.unit('2m^2')
area

2 m²

In [21]:
print(udr.yaml.dump(area))

!unit 2 m ** 2
...



# Class example

In [26]:
import attr
from unitdoc import UnitDocRegistry

udr = UnitDocRegistry()

In [27]:
@udr.serialize()   
@attr.s()
class Battery(object):
    name = attr.ib()

    weight = udr.attrib(default='45g')
    volume = udr.attrib(default='16ml', default_unit='ml')
    capacity = udr.attrib(default='3.0Ah')
    voltage = udr.attrib(default='3.6V', description ='Average voltage')

In [28]:
a_battery = Battery(name = 'battery', weight='43g')
print(a_battery)

Battery(name='battery', weight=<Quantity(43, 'gram')>, volume=<Quantity(16, 'milliliter')>, capacity=<Quantity(3.0, 'Ah')>, voltage=<Quantity(3.6, 'volt')>)


In [29]:
# You can do any operation that are allowed by the pint package
energy = (a_battery.capacity * a_battery.voltage).to('Wh')
energy_density = (energy / a_battery.weight).to('Wh/kg')
print(f'{energy} @  {energy_density}')

10.8 Wh @  251.2 Wh / kg


In [30]:
fn = 'a_battery.yaml'

# look at serialized form
print(a_battery.serialize())

# save to yaml file
with open(fn, 'w') as f:
    f.write(a_battery.serialize())

# load from yaml file
with open(fn, 'r') as f:
    a_loaded_battery = Battery.deserialize(f.read())


name: battery
weight: !unit 43 g
volume: !unit 16 ml
capacity: !unit 3 Ah
voltage: !unit 3.6 V



In [31]:
assert a_battery == a_loaded_battery

# Related packages

In [66]:
q = udr.ureg('1000gram').to('kg')
print(q)

1 kg


In [71]:
q_yaml = udr.yaml.dump(dict(weight=q))
print(q_yaml)

weight: !unit 1 kg



In [72]:
udr.cattr.unstructure(a_battery)

{'capacity': 3.0 Ah,
 'name': 'battery',
 'voltage': 3.6 V,
 'volume': 16 ml,
 'weight': 43 g}

In [78]:
@udr.serialize()   
@attr.s()
class Thing(object):
    weight = udr.attrib(default='45g', description ='Total weight')

a_thing = Thing()
a_thing_dict = udr.cattr.unstructure(a_thing)
assert type(a_thing_dict) == dict

print(a_thing_dict['weight'])


45 g


# Features

In [1]:
import attr
from unitdoc import UnitDocRegistry, DimensionalityError

udr = UnitDocRegistry()

In [87]:
@udr.serialize()   
@attr.s()
class Battery(object):
    name = attr.ib()

    weight = udr.attrib(default='45g')
    volume = udr.attrib(default='16ml', default_unit='ml')
    capacity = udr.attrib(default='3.0Ah')
    voltage = udr.attrib(default='3.6V', description ='Average voltage')

In [88]:
# Quantities are automatically normalized to a given default_unit:
a_battery = Battery(name = 'battery', volume='15903 mm^3')
print(a_battery.volume)

15.9 ml


In [89]:
# Incompatible units raise DimensionalityError
from unitdoc import DimensionalityError

try:
    a_battery = Battery(name = 'battery', volume='42 g')
except DimensionalityError as e:
    print(e)

Cannot convert from 'gram' ([mass]) to 'milliliter' ([length] ** 3)


In [90]:
# Retrieve description of parameters
from unitdoc import get_attr_description
print(get_attr_description(a_battery.__class__, 'voltage'))

Average voltage


In [91]:
# Quantities are automatically normalized to a given default_unit:
a_battery = Battery(name = 'battery', volume=None)
print(a_battery.volume)

None


In [92]:
print(a_battery.serialize())

name: battery
weight: !unit 45 g
volume:
capacity: !unit 3 Ah
voltage: !unit 3.6 V



In [97]:
a_battery = Battery(name = 'battery')
a_battery.volume = 99
type(a_battery.volume)

int

In [99]:
@udr.serialize()   
@attr.s(kw_only=True, frozen=True)
class BetterBattery(object):
    name = attr.ib()

    weight = udr.attrib(default='45g')
    volume = udr.attrib(default='16ml', default_unit='ml')
    capacity = udr.attrib(default='3.0Ah')
    voltage = udr.attrib(default='3.6V', description ='Average voltage')

In [100]:
# The following line fails with a TypeError
# This is good, because positional arguments can be dangerous when data model changes over time.
a_battery = BetterBattery('battery', '42g', '16ml') 

TypeError: __init__() takes 1 positional argument but 4 were given

In [102]:
# The kw_only flag enforces to use keywords in the constructor
# This makes the code more save and understandable
a_battery = BetterBattery(name='battery', weight='42g', volume='16ml') 

In [103]:
# Fails with Frozen InstanceError, which avoids unchecked override of attributes
a_battery.volume = 99 

FrozenInstanceError: 

In [104]:
# Update values correctly, including unit conversion and checks
a_battery = attr.evolve(a_battery, volume='12cm^3')
print(a_battery.volume)

12 ml
